# Coursera Capstone Project

## São Paulo Best place for a new Coffee Shop

Is it possible to determine the best place to start a new Specialty Coffee Shop business?

According to the Specialty Coffee Association of America (SCAA), coffee which scores 80 points or above on a 100-point scale is graded "specialty."

Brazil is the top coffee producer nation and one of the top 15 consumer.

I´ll provide a list of some of the TOP Specialty Coffee Shops in São Paulo, based on Foursquare information I´ll plot this locations on a map.
As Specialty Coffee usualy costs higher them regular coffee, using information of regions human development index (HDI) and per capita income, I´ll check if it´s possible to determine the better location for a new business.

In [1]:
import requests
import lxml.html as lh
import pandas as pd

Importing list of Coffee Shops that work with specialty coffee

In [24]:
df = pd.read_csv("cafeterias_especiais.csv", delimiter=';')
df.head()

,Name,Address,Neighborhood
0,COFFEE LAB,"Rua Fradique Coutinho, 1340",Vila Madalena
1,UM COFFEE CO.,"Rua Júlio Conceição, 553",Bom Retiro
2,UM COFFEE CO.,"Rua Iaiá, 62",Itaim Bibi
3,POR UM PUNHADO DE DÓLARES,"Rua Nestor Pestana, 115",Consolação
4,URBE CAFÉ,"Rua Antônio Carlos, 404",Consolação


In [23]:
#df.set_index('Neighborhood', inplace=True)

Importing human development index (HDI) list

In [16]:
idh = pd.read_csv("IDH.csv", delimiter=';')
idh.head()

,Posicao,Distrito,IDH
0,1,Moema,"0,981"
1,2,Pinheiros,"0,98"
2,3,Perdizes,"0,977"
3,4,Jardim Paulista,"0,975"
4,5,Alto de Pinheiros,"0,972"


In [17]:
idh.drop('Posicao', axis=1, inplace=True)

In [18]:
idh.rename(index=str, columns={'Distrito ' : 'Distrito'}, inplace=True)

In [19]:
idh['Distrito'] = idh['Distrito'].str.strip()

In [20]:
#idh.set_index('Distrito', inplace=True)

In [21]:
idh.columns

Index(['Distrito', 'IDH'], dtype='object')

In [22]:
idh.head()

,Distrito,IDH
0,Moema,"0,981"
1,Pinheiros,"0,98"
2,Perdizes,"0,977"
3,Jardim Paulista,"0,975"
4,Alto de Pinheiros,"0,972"


Putting all together.

In [35]:
final_result= df.set_index('Neighborhood').join(idh.set_index('Distrito'))
final_result.reset_index(inplace=True)

In [41]:
final_result.rename(index=str, columns={'index':'Bairro'}, inplace=True)

In [42]:
final_result = final_result.append({'Bairro' : 'Vila Madalena', 'Name' : 'COFFEE LAB' , 'Address' : 'Rua Fradique Coutinho, 1340', 'IDH' : '0,960'} , ignore_index=True)

In [43]:
final_result = final_result.append({'Bairro' : 'Vila Madalena', 'Name' : 'COFFEE LAB' , 'Address' : 'Rua Wisard, 171', 'IDH' : '0,960'}, ignore_index=True)

In [44]:
final_result

,Bairro,Name,Address,IDH
0,Bom Retiro,UM COFFEE CO.,"Rua Júlio Conceição, 553","0,847"
1,Consolação,POR UM PUNHADO DE DÓLARES,"Rua Nestor Pestana, 115","0,965"
2,Consolação,URBE CAFÉ,"Rua Antônio Carlos, 404","0,965"
3,Itaim Bibi,UM COFFEE CO.,"Rua Iaiá, 62","0,971"
4,Moema,FREAK CAFÉ,"Avenida Jurema, 359","0,981"
5,Pinheiros,SOFÁ CAFÉ,"Rua Artur de Azevedo, 514","0,98"
6,Pinheiros,SOFÁ CAFÉ,"Rua Bianchi Bertoldi, 130","0,98"
7,Pinheiros,KING OF THE FORK,"Rua Artur de Azevedo, 1317","0,98"
8,Pinheiros,TORRA CLARA,"Rua Oscar Freire, 2.286","0,98"
9,Vila Madalena,COFFEE LAB,"Rua Fradique Coutinho, 1340",NaN


In [45]:
final_result.dropna(axis=0, inplace=True)

In [51]:
final_result.reset_index(inplace=True)

In [53]:
final_result.drop('index', axis=1, inplace=True)

In [54]:
final_result

,Bairro,Name,Address,IDH
0,Bom Retiro,UM COFFEE CO.,"Rua Júlio Conceição, 553","0,847"
1,Consolação,POR UM PUNHADO DE DÓLARES,"Rua Nestor Pestana, 115","0,965"
2,Consolação,URBE CAFÉ,"Rua Antônio Carlos, 404","0,965"
3,Itaim Bibi,UM COFFEE CO.,"Rua Iaiá, 62","0,971"
4,Moema,FREAK CAFÉ,"Avenida Jurema, 359","0,981"
5,Pinheiros,SOFÁ CAFÉ,"Rua Artur de Azevedo, 514","0,98"
6,Pinheiros,SOFÁ CAFÉ,"Rua Bianchi Bertoldi, 130","0,98"
7,Pinheiros,KING OF THE FORK,"Rua Artur de Azevedo, 1317","0,98"
8,Pinheiros,TORRA CLARA,"Rua Oscar Freire, 2.286","0,98"
9,Vila Madalena,COFFEE LAB,"Rua Fradique Coutinho, 1340","0,960"


Now let´s donwload the lat and long of our Specialty Coffee Shops

In [55]:
latlong = pd.DataFrame({'Lat':[], 'Long':[]})
latlong

,Lat,Long


Creating Lat Long of Coffee Shops list

In [56]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

for i in final_result.Address:
    address = i
    geolocator = Nominatim(user_agent="sp_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    latlong = latlong.append({'Lat': latitude, 'Long': longitude}, ignore_index=True)
    #print('The geograpical coordinate of {} in Sao Paulo are {}, {}.'.format((i), latitude, longitude))

latlong

,Lat,Long
0,-23.527525,-46.641472
1,-23.548556,-46.645346
2,-22.877471,-43.580906
3,-23.586935,-46.679292
4,-23.611181,-46.655530
5,-23.559819,-46.677229
6,-23.568654,-46.690946
7,-23.563975,-46.683830
8,-23.567231,-46.664316
9,-23.556041,-46.691383


Putting all together

In [57]:
result = pd.concat([final_result, latlong], axis=1)
result

,Bairro,Name,Address,IDH,Lat,Long
0,Bom Retiro,UM COFFEE CO.,"Rua Júlio Conceição, 553","0,847",-23.527525,-46.641472
1,Consolação,POR UM PUNHADO DE DÓLARES,"Rua Nestor Pestana, 115","0,965",-23.548556,-46.645346
2,Consolação,URBE CAFÉ,"Rua Antônio Carlos, 404","0,965",-22.877471,-43.580906
3,Itaim Bibi,UM COFFEE CO.,"Rua Iaiá, 62","0,971",-23.586935,-46.679292
4,Moema,FREAK CAFÉ,"Avenida Jurema, 359","0,981",-23.611181,-46.655530
5,Pinheiros,SOFÁ CAFÉ,"Rua Artur de Azevedo, 514","0,98",-23.559819,-46.677229
6,Pinheiros,SOFÁ CAFÉ,"Rua Bianchi Bertoldi, 130","0,98",-23.568654,-46.690946
7,Pinheiros,KING OF THE FORK,"Rua Artur de Azevedo, 1317","0,98",-23.563975,-46.683830
8,Pinheiros,TORRA CLARA,"Rua Oscar Freire, 2.286","0,98",-23.567231,-46.664316
9,Vila Madalena,COFFEE LAB,"Rua Fradique Coutinho, 1340","0,960",-23.556041,-46.691383


Now let´s download out map and start to configure Foursquare API

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes

import folium # map rendering library

In [ ]:
# create map of SaoPaulo using latitude and longitude values
map_sp = folium.Map(location=[latitude, longitude], zoom_start=14)

map_sp

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'LBSKSHMKF53O1YGE4SQLCOTYBZU4ZKN4V11KJI1G5WEBN3KR' # your Foursquare ID
CLIENT_SECRET = '0ERKFMXE3KMZBJS3VOQA20H5AUWW4JGKVRU1GE0130ZGABCM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Search for Venues

In [ ]:
limit = 10 # limit of number of venues returned by Foursquare API
radius = 250 # define radius
#categoryId = '4bf58dd8d48988d1e0931735' #Coffee Shop Category ID
llAcc = 100
ll = -23.556041,-46.691383
query = 'Coffee Lab'
# create URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
#    categoryId,
    query,
    limit)
url # display URL

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))